Problem Statement

Digital payment platforms rely heavily on the sustained activity of merchants to drive transaction volume and revenue. However, a significant proportion of merchants tend to become inactive shortly after onboarding, leading to lost revenue and inefficient acquisition spend.

The objective of this project is to identify merchants who are at risk of churning by analyzing their historical transaction behavior. Using transactional data, we aim to build a machine learning model that can predict whether a merchant is likely to stop transacting within a defined future period. Early identification of such merchants can enable proactive retention interventions, such as targeted support or incentives.

This project formulates merchant churn as a binary classification problem, where past transaction patterns are used to predict future inactivity.

Important Assumption:
We treat the dataset as historical transaction activity from a payment platform up to a fixed cut-off date. Merchants are considered churned if they show no activity within a defined period preceding this date.


Problem being solved

Merchants stop transacting without warning (merchant churn)

Churn leads to revenue loss and wasted acquisition cost

Platforms often react after churn happens

What ML helps with:

Detects early signals of inactivity

Flags merchants before they fully churn

Prioritizes retention efforts where impact is highest

Why this matters for Razorpay-like companies

Improves merchant lifetime value

Reduces support and acquisition inefficiency

Enables data-driven retention instead of guesswork

In [1]:
import pandas as pd

In [3]:
filepath = "D:/Razorpay/DataSet/Online Retail.xlsx"

df = pd.read_excel(filepath)

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [10]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [13]:
df.drop('Description', axis=1, inplace=True)

In [14]:
df

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...
541904,581587,22613,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,4,2011-12-09 12:50:00,4.15,12680.0,France


In [15]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [18]:
df.dropna(inplace=True)

In [20]:
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Defining Churn:

A merchant is churned if they have no transactions in the last 30 days of the dataset.

What we are assuming

The dataset represents transaction activity on a payment platform (proxy for Razorpay).

Each CustomerID is treated as a business/merchant using that platform.

InvoiceDate represents points in time when that merchant was active (processed payments).

latest_date is the cut-off date up to which the data was extracted.

churn = 1 if days > 30

churn = 0 if days < 30

In [25]:
latest_date = df['InvoiceDate'].max()
latest_date

Timestamp('2011-12-09 12:50:00')

In [27]:
latest_date_per_merchant = df.groupby('CustomerID')['InvoiceDate'].max()

In [29]:
latest_date_per_merchant

# We're referencing the invoice date with the customer id, here we're assuming that despite the 
# customer being a business or a customer he/she is using razorpay to make its payment to the business.

CustomerID
12346.0   2011-01-18 10:17:00
12347.0   2011-12-07 15:52:00
12348.0   2011-09-25 13:13:00
12349.0   2011-11-21 09:51:00
12350.0   2011-02-02 16:01:00
                  ...        
18280.0   2011-03-07 09:52:00
18281.0   2011-06-12 10:53:00
18282.0   2011-12-02 11:43:00
18283.0   2011-12-06 12:02:00
18287.0   2011-10-28 09:29:00
Name: InvoiceDate, Length: 4372, dtype: datetime64[ns]

In [58]:
latest_date_per_merchant.info()

<class 'pandas.core.series.Series'>
Index: 4372 entries, 12346.0 to 18287.0
Series name: InvoiceDate
Non-Null Count  Dtype         
--------------  -----         
4372 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 68.3 KB


In [32]:
time_delta = latest_date - latest_date_per_merchant

time_delta

CustomerID
12346.0   325 days 02:33:00
12347.0     1 days 20:58:00
12348.0    74 days 23:37:00
12349.0    18 days 02:59:00
12350.0   309 days 20:49:00
                 ...       
18280.0   277 days 02:58:00
18281.0   180 days 01:57:00
18282.0     7 days 01:07:00
18283.0     3 days 00:48:00
18287.0    42 days 03:21:00
Name: InvoiceDate, Length: 4372, dtype: timedelta64[ns]

In [46]:
churn = (time_delta.dt.days > 30).astype(int)

churn

CustomerID
12346.0    1
12347.0    0
12348.0    1
12349.0    0
12350.0    1
          ..
18280.0    1
18281.0    1
18282.0    0
18283.0    0
18287.0    1
Name: InvoiceDate, Length: 4372, dtype: int32

In [52]:
per_merchant_churn_table = pd.DataFrame({
    "InactivityDays": time_delta.dt.days,
    "Churn": churn
})

per_merchant_churn_table.head()

,InactivityDays,Churn
CustomerID,,
12346.0,325,1
12347.0,1,0
12348.0,74,1
12349.0,18,0
12350.0,309,1


In [60]:
# Sanity Check
per_merchant_churn_table['Churn'].value_counts()

Churn
1    2630
0    1742
Name: count, dtype: int64

In [61]:
# Sanity Check Again
per_merchant_churn_table["Churn"].value_counts(normalize=True)

Churn
1    0.601555
0    0.398445
Name: proportion, dtype: float64

We now decide which past data is allowed to be used as features to predict churn.
Features must come from the past, not from the churn window.

What “features” means here:

Features = measurable characteristics of merchant behavior
Numbers the ML model will use to learn patterns.

In plain terms:

Features describe how a merchant behaved before we decide if they churned.

Examples:

Number of transactions

Average transaction value

Days since last transaction

Trend in activity (up/down)

Each merchant → one row of numbers.

What “past data” means

“Past data” means:

Transaction history before the churn decision point

Data that would realistically be available at prediction time

In our setup:

Last 30 days → used only to decide churn

Before that → safe to use as features

So “past data” = transactions outside the churn window.

Windows

Churn window:
T − 30 days → T
→ used only to label churn (already done)

Observation window (features):
T − 120 days → T − 30 days
(i.e., 90 days of behavior)

In [68]:
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [69]:
latest_date

Timestamp('2011-12-09 12:50:00')

In [71]:
# Filter transactions before churn off start period: latest_date - 30 days

cutoff_date = latest_date - pd.Timedelta(days=30)

print(cutoff_date)


2011-11-09 12:50:00


In [75]:
start_date = latest_date - pd.Timedelta(days=120)

print(start_date)

2011-08-11 12:50:00


In [85]:
obs_df = df[(df['InvoiceDate'] >= start_date) & (df['InvoiceDate'] <= cutoff_date)]

obs_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
298384,563018,16169E,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom
298385,563018,23263,12,2011-08-11 12:53:00,1.25,13427.0,United Kingdom
298386,563018,23546,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom
298387,563018,23547,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom
298388,563018,22178,24,2011-08-11 12:53:00,1.25,13427.0,United Kingdom


Turn the 90-day observation window (obs_df) into numerical features per merchant that summarize behavior.

One row per CustomerID

Multiple feature columns

Ready to be joined with the churn labels later

What counts as a “feature” here

Think: How did the merchant behave in the last 90 days?

Core feature groups (we’ll start with these)

Per merchant (CustomerID)

1. Transaction frequency

txn_count_90d
→ How many transactions in the last 90 days

2️. Monetary value

total_txn_value_90d
→ Total value of transactions

avg_txn_value_90d
→ Average transaction value

3. Recency (within observation window)

recency_days
→ Days since the merchant’s last transaction before cutoff

In [87]:
obs_df

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
298384,563018,16169E,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom
298385,563018,23263,12,2011-08-11 12:53:00,1.25,13427.0,United Kingdom
298386,563018,23546,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom
298387,563018,23547,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom
298388,563018,22178,24,2011-08-11 12:53:00,1.25,13427.0,United Kingdom
...,...,...,...,...,...,...,...
452020,575312,22083,6,2011-11-09 12:49:00,2.95,13588.0,United Kingdom
452021,575312,23355,4,2011-11-09 12:49:00,4.95,13588.0,United Kingdom
452022,575312,22110,6,2011-11-09 12:49:00,2.55,13588.0,United Kingdom
452023,575312,22037,12,2011-11-09 12:49:00,0.42,13588.0,United Kingdom


In [97]:
# Tx Frequency

tx90 = obs_df.groupby('CustomerID')['InvoiceNo'].nunique()

tx90.value_counts()

InvoiceNo
1     1317
2      598
3      267
4      147
5       98
6       60
7       38
8       15
9       14
10      10
11       8
12       8
13       3
15       3
24       3
14       2
29       2
26       2
19       1
21       1
90       1
25       1
48       1
16       1
53       1
17       1
30       1
Name: count, dtype: int64

In [102]:
obs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125737 entries, 298384 to 452024
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    125737 non-null  object        
 1   StockCode    125737 non-null  object        
 2   Quantity     125737 non-null  int64         
 3   InvoiceDate  125737 non-null  datetime64[ns]
 4   UnitPrice    125737 non-null  float64       
 5   CustomerID   125737 non-null  float64       
 6   Country      125737 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 7.7+ MB


In [107]:
#  Monetary Value - total value of txs

obs_df['TotalPrice'] = obs_df['Quantity'] * obs_df['UnitPrice']

obs_df.head()

C:\Users\gamef\AppData\Local\Temp\ipykernel_34992\2662274485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_df['TotalPrice'] = obs_df['Quantity'] * obs_df['UnitPrice']


,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
298384,563018,16169E,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom,10.5
298385,563018,23263,12,2011-08-11 12:53:00,1.25,13427.0,United Kingdom,15.0
298386,563018,23546,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom,10.5
298387,563018,23547,25,2011-08-11 12:53:00,0.42,13427.0,United Kingdom,10.5
298388,563018,22178,24,2011-08-11 12:53:00,1.25,13427.0,United Kingdom,30.0


In [110]:
total_value_90 = obs_df.groupby('CustomerID')['TotalPrice'].sum()

total_value_90

CustomerID
12347.0    1294.32
12348.0     310.00
12352.0     944.23
12357.0    6207.67
12359.0    2876.85
            ...   
18276.0     335.86
18277.0     110.38
18278.0     173.90
18283.0     249.55
18287.0    1072.00
Name: TotalPrice, Length: 2604, dtype: float64

In [124]:
# Avg Tx Value
avg_value_90 = (obs_df.groupby(
    ['CustomerID', 'InvoiceNo'])['TotalPrice']
    .sum()
    .groupby('CustomerID')
    .mean()
)
avg_value_90

CustomerID
12347.0    1294.320000
12348.0     310.000000
12352.0     314.743333
12357.0    6207.670000
12359.0    2876.850000
              ...     
18276.0     335.860000
18277.0     110.380000
18278.0     173.900000
18283.0     124.775000
18287.0     536.000000
Name: TotalPrice, Length: 2604, dtype: float64

In [125]:
# recency_days
# Days since the merchant’s last transaction before cutoff

recency_days = (
    obs_df
    .groupby('CustomerID')['InvoiceDate']
    .max()
    .apply(lambda x: (cutoff_date - x).days)
)

recency_days

CustomerID
12347.0     9
12348.0    44
12352.0     5
12357.0     2
12359.0    27
           ..
18276.0    13
18277.0    27
18278.0    43
18283.0    12
18287.0    12
Name: InvoiceDate, Length: 2604, dtype: int64

In [126]:
# Assembing into one Feature df

feature_df = pd.DataFrame({
    "TxFreq": tx90,
    "TotalTx90": total_value_90,
    "AvgTx90": avg_value_90,
    "Recency": recency_days
})

feature_df.head()

,TxFreq,TotalTx90,AvgTx90,Recency
CustomerID,,,,
12347.0,1,1294.32,1294.320000,9
12348.0,1,310.00,310.000000,44
12352.0,3,944.23,314.743333,5
12357.0,1,6207.67,6207.670000,2
12359.0,1,2876.85,2876.850000,27


In [127]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2604 entries, 12347.0 to 18287.0
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TxFreq     2604 non-null   int64  
 1   TotalTx90  2604 non-null   float64
 2   AvgTx90    2604 non-null   float64
 3   Recency    2604 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 101.7 KB
